In [1]:
import configparser
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sqlalchemy
import sqlite3

import sqlite_utils

%load_ext sql

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
plt.style.use('raph-base')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 30)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

np.random.seed(666)

In [3]:
print(sys.executable)
print(sys.version)
print(f'Pandas {pd.__version__}')

C:\Users\r2d4\miniconda3\envs\py3\python.exe
3.8.3 (default, May 19 2020, 06:50:17) [MSC v.1916 64 bit (AMD64)]
Pandas 1.1.3


## Connect to DB

- With sqlite you can connect very lightweight to your DB (see first cell below). 
- But to be able to use sqlalchemy or ipython-sql you need a 'proper' connection string, see second cell.

In [4]:
# Connect to DiscoBase.db the sqlite way

db_path = "DiscoBase.db"
conn = sqlite3.connect(db_path, isolation_level=None)  # auto commit mode
cur = conn.cursor()

conn, cur

(<sqlite3.Connection at 0x24e2d5978a0>, <sqlite3.Cursor at 0x24e2d6b8c00>)

In [5]:
# Create a connection string and engine for sqlalchemy and ipython-sql

sqlite_conn_str = f"sqlite:///{db_path}"

sqlite_conn_str

'sqlite:///DiscoBase.db'

In [6]:
# Connect to sqlalchemy

engine = sqlalchemy.create_engine(sqlite_conn_str)

engine

Engine(sqlite:///DiscoBase.db)

In [7]:
# Connect to ipython-sql

%sql $sqlite_conn_str

'Connected: @DiscoBase.db'

## Query DB

I show 5 of the possible ways to query the DB
1. using ipython-sql
2. using sqlite functionality 'raw'
3. using my query_read wrapper
4. using pandas with sqlite connection
5. using pandas with sqlalchemy engine


In [8]:
query = (
    """
    SELECT * 
      FROM albums 
      WHERE genre = 'Death Metal'
      LIMIT 3
    ;"""
)

In [9]:
# 1. Read using sqlite connection
# Returns a cursor containing the data tuples

%sql $query

 * sqlite:///DiscoBase.db
Done.


index,Alive,Artist,Album,Format,Year,Genre,Price,Purchase_Date,Vinyl_Color,Lim_Edition,Number,Label,Digitized,Remarks,Rating
0,1.0,Dismember,Pieces,"12""",1992.0,Death Metal,35.0,1992-01-01 00:00:00,None,None,None,None,Nein,None,None
1,1.0,Dismember,Skin Her Alive,"7"" Pic",1991.0,Death Metal,15.0,1992-01-01 00:00:00,None,None,None,None,Nein,None,None
2,1.0,Dismember,Like An Everflowing Stream,LP,1991.0,Death Metal,155.0,1992-01-01 00:00:00,None,None,None,None,Nein,None,None


In [10]:
# 2. Read using sqlite connection
# Returns a cursor containing the data tuples

with conn:
    data = conn.execute(query)
    for row in data:
        print(row)

(0, 1.0, 'Dismember', 'Pieces', '12"', 1992.0, 'Death Metal', 35.0, '1992-01-01 00:00:00', None, None, None, None, 'Nein', None, None)
(1, 1.0, 'Dismember', 'Skin Her Alive', '7" Pic', 1991.0, 'Death Metal', 15.0, '1992-01-01 00:00:00', None, None, None, None, 'Nein', None, None)
(2, 1.0, 'Dismember', 'Like An Everflowing Stream', 'LP', 1991.0, 'Death Metal', 155.0, '1992-01-01 00:00:00', None, None, None, None, 'Nein', None, None)


In [11]:
# 3. Read using cursor and my utils function
# Returns a list of tuples

print(sqlite_utils.query_read(query, cur))

[(0, 1.0, 'Dismember', 'Pieces', '12"', 1992.0, 'Death Metal', 35.0, '1992-01-01 00:00:00', None, None, None, None, 'Nein', None, None), (1, 1.0, 'Dismember', 'Skin Her Alive', '7" Pic', 1991.0, 'Death Metal', 15.0, '1992-01-01 00:00:00', None, None, None, None, 'Nein', None, None), (2, 1.0, 'Dismember', 'Like An Everflowing Stream', 'LP', 1991.0, 'Death Metal', 155.0, '1992-01-01 00:00:00', None, None, None, None, 'Nein', None, None)]


In [12]:
# 4. Read with pandas, using sqlite connection
# Returns a dataframe

pd.read_sql(query, conn, index_col="index")

,Alive,Artist,Album,Format,Year,Genre,Price,Purchase_Date,Vinyl_Color,Lim_Edition,Number,Label,Digitized,Remarks,Rating
index,,,,,,,,,,,,,,,
0,1.00,Dismember,Pieces,"12""","1,992.00",Death Metal,35.00,1992-01-01 00:00:00,None,None,None,None,Nein,None,None
1,1.00,Dismember,Skin Her Alive,"7"" Pic","1,991.00",Death Metal,15.00,1992-01-01 00:00:00,None,None,None,None,Nein,None,None
2,1.00,Dismember,Like An Everflowing Stream,LP,"1,991.00",Death Metal,155.00,1992-01-01 00:00:00,None,None,None,None,Nein,None,None


In [13]:
# 5. Read with pandas, using sqlalchemy engine
# Returns a dataframe

pd.read_sql(query, conn, index_col="index")

,Alive,Artist,Album,Format,Year,Genre,Price,Purchase_Date,Vinyl_Color,Lim_Edition,Number,Label,Digitized,Remarks,Rating
index,,,,,,,,,,,,,,,
0,1.00,Dismember,Pieces,"12""","1,992.00",Death Metal,35.00,1992-01-01 00:00:00,None,None,None,None,Nein,None,None
1,1.00,Dismember,Skin Her Alive,"7"" Pic","1,991.00",Death Metal,15.00,1992-01-01 00:00:00,None,None,None,None,Nein,None,None
2,1.00,Dismember,Like An Everflowing Stream,LP,"1,991.00",Death Metal,155.00,1992-01-01 00:00:00,None,None,None,None,Nein,None,None


In [14]:
%sql SELECT * FROM albums WHERE artist = "Organic"

 * sqlite:///DiscoBase.db
Done.


index,Alive,Artist,Album,Format,Year,Genre,Price,Purchase_Date,Vinyl_Color,Lim_Edition,Number,Label,Digitized,Remarks,Rating
291,1.0,Organic,Carved In Flesh,LP,2018.0,Hardcore,20.0,2020-11-01 00:00:00,None,None,None,Testimony Records,Nein,None,None
